In [1]:
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from itertools import chain
from time import sleep

import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from tqdm import tqdm

In [3]:
all_results = []
as_csv = "block,form_id,responder_uri,study_entry_ids\n"

In [4]:
SERVICE_ACCOUNT_FILE = "../../research-430307-ad3438ad46d0.json"
APPROVAL_URL = "https://app.prolific.com/submissions/complete?cc=CWYHF4HQ"
SCOPES = ["https://www.googleapis.com/auth/forms", "https://www.googleapis.com/auth/drive"]

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

forms_service = build("forms", "v1", credentials=credentials)
drive_service = build("drive", "v3", credentials=credentials)

data = pd.read_csv("../../generated-dataset-30_000/human-study/open/study_data_open.csv")

In [10]:
BLOCK_SIZE = 20
TOTAL_NUMBER = 820  # len(data)
assert TOTAL_NUMBER % BLOCK_SIZE == 0


def create_form(block, form_data):
    # Creates the initial form
    title = f"QuAnTS Motion Question Answering #{block} (open)"
    result = (
        forms_service.forms()
        .create(
            body={
                "info": {"title": title},
            }
        )
        .execute()
    )
    form_id = result["formId"]

    list_of_actions = [
        "playing guitar",
        "punching",
        "running",
        "jumping once",
        "drinking with the left hand",
        "bowing",
        "throwing a ball",
        "shaking hands",
        "skipping rope",
        "picking something up with both hands",
        "dancing",
        "eating with the right hand",
        "waving",
        "catching a ball",
        "kicking a ball",
        "sitting down",
        "golfing (swinging a club)",
        "T-posing",
        "holding a baby",
    ]
    formatted_list_of_actions = "\n".join(f"- {action}" for action in list_of_actions)

    num_entries_on_page = 4
    num_actions = 4
    form_content_update = {
        "includeFormInResponse": False,
        "requests": [
            {
                "updateFormInfo": {
                    "info": {
                        "title": title,
                        "documentTitle": title,
                        "description": f"""This task is about question answering based on a short clip of a person performing four consecutive actions.

The depicted person performs exactly four (possibly equal) actions chosen from a pool of these actions (you don't need to remember them all):

{formatted_list_of_actions}

The four actions start every 4:00 seconds, resulting in:

- Seconds 00 to 04: activity #1
- Seconds 04 to 08: activity #2
- Seconds 08 to 12: activity #3
- Seconds 12 to 16: activity #4

The video will include the timestamps at the bottom.

Important note: If actions are repeated, they are considered to be one continuous action. For example, given the following sequence:

- Seconds 00 to 04: Bowing
- Seconds 04 to 08: Dancing
- Seconds 08 to 12: Dancing
- Seconds 12 to 16: Holding a baby

Then, for the question "How many times does the person dance?", the answer is "The person dances once.", not twice. If the question instead were "Was dancing performed the fewest times in the sequence?", the answer would similarly be "Yes"/"That is correct.", since all three distinct actions are performed once.

Please watch each video clip carefully and then answer the question based on your observations. If necessary, feel free to watch it again. It will loop until you submit your answer.

The answer should be given as a short but complete sentence with correct punctuation. There will only be 20 questions in total.""",
                    },
                    "updateMask": {
                        "paths": ["title", "documentTitle", "description"],
                    },
                },
            },
            *chain(
                [
                    {
                        "createItem": {
                            "item": {
                                "pageBreakItem": {},
                            },
                            "location": {"index": (num_entries_on_page * i) + 0},
                        },
                    },
                    # {
                    #     "createItem": {
                    #         "item": {
                    #             "title": "Reminder of the possible action types:",
                    #             "description": formatted_list_of_actions,
                    #             "textItem": {},
                    #         },
                    #         "location": {"index": (num_entries_on_page * i) + 1},
                    #     },
                    # },
                    {
                        "createItem": {
                            "item": {
                                "title": f"Question #{row['study_entry_id'] + 1}",
                                "imageItem": {
                                    "image": {"sourceUri": row["url"]},
                                },
                            },
                            "location": {"index": (num_entries_on_page * i) + 1},
                        },
                    },
                    # {
                    #                         "createItem": {
                    #                             "item": {
                    #                                 "title": "As a first step, please state the four actions you observed in the video. Please select '<not recognized>' if you cannot recognize it at all.",
                    #                                 "description": f"""All possible actions are listed here.
                    # {formatted_list_of_actions}""",
                    #                                 "textItem": {},
                    #                             },
                    #                             "location": {"index": (num_entries_on_page * i) + 2},
                    #                         },
                    #                     },
                    # *(
                    #     {
                    #         "createItem": {
                    #             "item": {
                    #                 "title": f"Action #{question_index}",
                    #                 "questionItem": {
                    #                     "question": {
                    #                         "required": True,
                    #                         # "questionId": f"{10*row['study_entry_id'] + question_index:08}",  # Doesn't work
                    #                         "choiceQuestion": {
                    #                             "type": "RADIO",
                    #                             "options": [
                    #                                 *({"value": action} for action in list_of_actions),
                    #                                 {"value": "?"},
                    #                             ],
                    #                             "shuffle": False,
                    #                         },
                    #                     }
                    #                 },
                    #             },
                    #             "location": {"index": (num_entries_on_page * i) + 2 + question_index},
                    #         },
                    #     }
                    #     for question_index in range(1, num_actions + 1)
                    # ),
                    {
                        "createItem": {
                            "item": {
                                "title": "Please first identify the four actions from in the video. Please select '<not recognized>' if you cannot recognize it at all.",
                                "questionGroupItem": {
                                    "questions": [
                                        {
                                            "required": True,
                                            "rowQuestion": {"title": f"Action #{question_index}"},
                                        }
                                        for question_index in range(1, num_actions + 1)
                                    ],
                                    "grid": {
                                        "columns": {
                                            "type": "RADIO",
                                            "options": [
                                                *({"value": action} for action in list_of_actions),
                                                {"value": "<not recognized>"},
                                            ],
                                        }
                                    },
                                },
                            },
                            "location": {"index": (num_entries_on_page * i) + 2},
                        }
                    },
                    {
                        "createItem": {
                            "item": {
                                "title": f"{row['question']} Please answer based on the actions you identified in FULL sentences.",
                                "questionItem": {
                                    "question": {
                                        "required": True,
                                        "questionId": f"{row['study_entry_id']:08}",
                                        # "choiceQuestion": {
                                        #     "type": "RADIO",
                                        #     "options": [
                                        #         # {"value": "Yes"},
                                        #         # {"value": "No"},
                                        #         {"value": "A"},
                                        #         {"value": "B"},
                                        #         {"value": "C"},
                                        #     ],
                                        #     "shuffle": False,
                                        # },
                                        "textQuestion": {"paragraph": True},
                                    }
                                },
                            },
                            "location": {"index": (num_entries_on_page * i) + 3},
                        },
                    },
                ]
                for i, row in form_data.iterrows()
            ),
            {
                "createItem": {
                    "item": {
                        "pageBreakItem": {},
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 0)},
                },
            },
            {
                "createItem": {
                    "item": {
                        "title": "Prolific ID",
                        "description": """Please enter your Prolific ID below, so we can complete and approve your submission. This is very important! You prolific ID consits of numbers and letters and is 24 characters long.""",
                        "questionItem": {
                            "question": {
                                "questionId": "999999",
                                "required": True,
                                "textQuestion": {"paragraph": False},
                            }
                        },
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 1)},
                },
            },
            {
                "createItem": {
                    "item": {
                        "pageBreakItem": {},
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 2)},
                },
            },
            {
                "createItem": {
                    "item": {
                        "title": "Done!",
                        "description": f"""You have now completed the task. Please click on the link below to finalize the task:

[Back to Prolific]({APPROVAL_URL}).""",
                        "textItem": {},
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 3)},
                },
            },
        ],
    }
    forms_service.forms().batchUpdate(formId=form_id, body=form_content_update).execute()

    # This does not work since the form is in the folder of the service account
    # drive_service.files().update(
    #     fileId=form_id,
    #     addParents="1yEu8OPKYru7c_4N0xcevr-8t-QQ1WUGT",
    #     # /QuAnTS/study/binary
    #     # , removeParents="root"
    #     fields="id, parents",
    # ).execute()

    # drive_service.permissions().create(
    #     fileId=form_id, body={"type": "user", "role": "writer", "emailAddress": PERSONAL_EMAIL}, fields="id"
    # ).execute()

    return form_id, result["responderUri"]


for block in tqdm(range(TOTAL_NUMBER // BLOCK_SIZE)):
    if any(res["block"] == block for res in all_results):
        print(f"Skipping block {block}")
        continue

    start_index = block * BLOCK_SIZE
    end_index = min(((block + 1) * BLOCK_SIZE), len(data))
    form_data = data[start_index:end_index].reset_index()

    form_id, responder_uri = create_form(block, form_data)
    print(f"Form #{block} created: {form_id}, {responder_uri}")

    all_results.append(
        {
            "block": block,
            "form_id": form_id,
            "responder_uri": responder_uri,
            "study_entry_ids": form_data["study_entry_id"].tolist(),
        }
    )
    as_csv += f"{block},\"{form_id}\",\"{responder_uri}\",\"{form_data['study_entry_id'].tolist()}\"\n"

    sleep(1)

all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv("../../generated-dataset-30_000/human-study/open/study_forms_open.csv", index=False)

  0%|          | 0/41 [00:00<?, ?it/s]

Skipping block 0
Skipping block 1
Skipping block 2
Skipping block 3
Skipping block 4
Skipping block 5
Skipping block 6
Skipping block 7
Skipping block 8
Skipping block 9
Skipping block 10
Skipping block 11
Skipping block 12
Skipping block 13
Skipping block 14
Skipping block 15
Skipping block 16
Form #17 created: 1lfZncS2K_Wae0-lyFi3xz6bUKEr3I5oG_oNY8fY7aFQ, https://docs.google.com/forms/d/e/1FAIpQLSeqRRCknHDl1TXecQJvSbcsXfmzS0J2ML6cI6hbvWRrHL-J6Q/viewform


 44%|████▍     | 18/41 [00:18<00:23,  1.01s/it]

Form #18 created: 1qTgeoNVYLFBFmPXfQY828sOqxEKBB4hI3OK3tUIJS70, https://docs.google.com/forms/d/e/1FAIpQLSd-mBdwHzg6L1DANRACx0IS0Cl8jq6E8Rvp69mNjwUWrdDcOw/viewform


 46%|████▋     | 19/41 [00:32<00:44,  2.02s/it]

...

 95%|█████████▌| 39/41 [05:12<00:27, 13.52s/it]

Form #39 created: 16EAOYsRcZYfY-VDcuN5PND3OfN_pea-ycpiDUZD77Pc, https://docs.google.com/forms/d/e/1FAIpQLSefaAE496tJmOZ_-SzhEqZAFSWMS8IsU2BKpZWux44f2AiWCQ/viewform


 98%|█████████▊| 40/41 [05:26<00:13, 13.78s/it]

Form #40 created: 1uQTmcw2B9mQ40fAfM8CkBELo5lp6LeSExPFrhIWq_XM, https://docs.google.com/forms/d/e/1FAIpQLScz3UVqSEy6WzAEzr2ed1uyh_n15c3fDSZ2fOFnJ8LZDz4ZFg/viewform


100%|██████████| 41/41 [05:38<00:00,  8.26s/it]


In [11]:
print(as_csv)

block,form_id,responder_uri,study_entry_ids
0,"1q8OcMyeytskMEdcdYm1sVSk0SjM2TeYv9NaFCyxMWKw","https://docs.google.com/forms/d/e/1FAIpQLSfQZey7BdjJuKTl0I4f5MlAQ5VE3qq1_Y6PlIONiEviC90GnA/viewform","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]"
1,"1d-d_SSV2M6gGifNgXRGjp7l4DYtXciH1DH_cQ-04Vvk","https://docs.google.com/forms/d/e/1FAIpQLSfT4GhiWjHPXs84Sl025eHbQSyWH6T8NGEhybH3RPBX5hMHUA/viewform","[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]"
2,"1hVn4XBHmvYKSm2exykbGSrC8rdgpOIauJCnVP3hMmCA","https://docs.google.com/forms/d/e/1FAIpQLSdH7Qwthfi1neBFHe5pcUL3w_6Q4les0HHYyqRF8RDfKSTy6g/viewform","[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]"
3,"1CdX9xtGjXYpespi2tjJXUCWqSG-m8AK6gP4fIQyoVtE","https://docs.google.com/forms/d/e/1FAIpQLSfLrkS158NmvV37rdaKDywXzOirFPW4CbAyE6UFfkKuI6j7Ug/viewform","[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]"
4,"1ZPEgyobJCiNLaW3sL0rSxrBG-r

In [12]:
print(all_results_df["responder_uri"].to_csv(index=False, header=False))

https://docs.google.com/forms/d/e/1FAIpQLSfQZey7BdjJuKTl0I4f5MlAQ5VE3qq1_Y6PlIONiEviC90GnA/viewform
https://docs.google.com/forms/d/e/1FAIpQLSfT4GhiWjHPXs84Sl025eHbQSyWH6T8NGEhybH3RPBX5hMHUA/viewform
https://docs.google.com/forms/d/e/1FAIpQLSdH7Qwthfi1neBFHe5pcUL3w_6Q4les0HHYyqRF8RDfKSTy6g/viewform
https://docs.google.com/forms/d/e/1FAIpQLSfLrkS158NmvV37rdaKDywXzOirFPW4CbAyE6UFfkKuI6j7Ug/viewform
https://docs.google.com/forms/d/e/1FAIpQLScm2cgSsXgQh8yh6kJBCAosy0CpME0QAhHZZhWzRIf2dVRkEw/viewform
https://docs.google.com/forms/d/e/1FAIpQLSecR4BRb1L87BbJ4ugz2pW_SpRr2V364Smqqu5cm9RlVt85Aw/viewform
https://docs.google.com/forms/d/e/1FAIpQLSecgCGXUYP1hdCSDj3L76U59uogHEfO5Wep6s-azF8GItObvA/viewform
https://docs.google.com/forms/d/e/1FAIpQLSfxsGZLf3edFELylzBhBIDSRYc_Olkc_Ht6pTOd26byTZFKlA/viewform
https://docs.google.com/forms/d/e/1FAIpQLScvj7iyHuY3yu1FSsiIelUsiAyAtUeSGt3VnA1P5y9AQ5zGMQ/viewform
https://docs.google.com/forms/d/e/1FAIpQLSfRVywYdg534gTQ8tU0ztwAlR4zIKG6lrCWY6ZSJmGVdKA4Nw/viewform


In [ ]:
all_form_ids = pd.concat(
    [
        pd.read_csv("../../generated-dataset-30_000/human-study/binary/study_forms_binary.csv")["form_id"],
        pd.read_csv("../../generated-dataset-30_000/human-study/multi/study_forms_multi.csv")["form_id"],
        pd.read_csv("../../generated-dataset-30_000/human-study/open/study_forms_open.csv")["form_id"],
    ]
)
len(all_form_ids)

In [ ]:
drive_service = build("drive", "v3", credentials=credentials)

all_forms = (
    drive_service.files().list(q="mimeType='application/vnd.google-apps.form'", pageSize=1000).execute()
)
ids = [entry["id"] for entry in all_forms["files"] if entry["id"] not in set(all_form_ids)]
len(all_forms["files"]), len(ids)

In [18]:
# for id in ids:
#     drive_service.files().delete(fileId=id).execute()